---

**Runtime Configuration:** This notebook has a paired setup script at `runtimes/fly_connectome_02_neuron_morphology_post_startup.sh` which provides the complete installation recipe for all dependencies. This script can be used as a post-startup script for Google Colab to automatically configure the environment.

---

## ⚠️ Setup Instructions for Google Colab

Before running this notebook, install dependencies using the runtime script.

**Recommended: Manual Installation (Run in first cell)**

```python
!wget https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/runtimes/fly_connectome_02_neuron_morphology_post_startup.sh
!bash fly_connectome_02_neuron_morphology_post_startup.sh
```


**Alternative: Automatic Post-Startup Script**

For repeated use, you can set this as a post-startup script:
1. Go to: **Runtime → Change runtime type → Advanced settings**
2. Under "Post-startup script", enter:
   ```
   https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/runtimes/fly_connectome_02_neuron_morphology_post_startup.sh
   ```
3. Click **Save** and start the runtime

**Memory:** Colab Pro (25-30 GB RAM) recommended for best performance.


In [ ]:
# Run this cell FIRST to install dependencies (Colab only)
!wget https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/runtimes/fly_connectome_02_neuron_morphology_post_startup.sh
!bash fly_connectome_02_neuron_morphology_post_startup.sh


# Core Tutorial

In [ ]:
# Download utils.py if not available (for Google Colab)
import os
if not os.path.exists('utils.py'):
    print('📥 Downloading utils.py from GitHub...')
    import urllib.request
    url = 'https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/utils.py'
    urllib.request.urlretrieve(url, 'utils.py')
    print('✓ Downloaded utils.py')
else:
    print('✓ utils.py already available')


## Introduction

The purpose of this tutorial is to examine neuron morphology.

We can visualise neurons, compare their morphological similarity with [NBLAST](https://github.com/navis-org/navis), and plot them with their synapses.

To do this, we are going to look at one of the data 'subsets' we have pre-prepared.

I have subset the synapses and edgelist relevant for a few interesting areas, so let's use one of those.

You can change which subset and dataset to examine by changing the variables in the setup cells below.

By default, let us look at the **front leg neuropil**!

This is available for BANC, maleCNS and MANC.


## Setup and Configuration

## Import Packages

We'll use:
- **navis**: Neuron analysis and visualization
- **pandas**: Data manipulation
- **pyarrow**: Reading Feather files
- **gcsfs**: Google Cloud Storage access
- **plotly**: Interactive 3D visualizations
- **trimesh**: 3D mesh processing

**Installation:**
```bash
pip install navis[all] flybrains gcsfs plotly kaleido trimesh
```

In [ ]:
# Environment detection and Colab setup (auto-configured)
try:
    import google.colab
    IN_COLAB = True
    
    # Colab setup
    
    # Authenticate
    from google.colab import auth
    auth.authenticate_user()
    print("✓ Authenticated with Google Cloud")
    
    # Download utils.py
    import urllib.request, os
    HELPER_URL = "https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/utils.py"
    if not os.path.exists("utils.py"):
        urllib.request.urlretrieve(HELPER_URL, "setup_helpers.py")
    
    print("✓ Colab environment ready\n")
except ImportError:
    IN_COLAB = False
    # Local environment - no output needed
    pass

In [ ]:
# Import standard packages
import pandas as pd
import numpy as np
import gcsfs
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import squareform
from tqdm import tqdm
import warnings
import re
warnings.filterwarnings('ignore')

# Import neuroscience packages
import navis
import trimesh

# Configure plotly renderer
try:
    import google.colab
    pio.renderers.default = "colab"
except ImportError:
    pio.renderers.default = "notebook"

# Import helper functions from utils module
from utils import construct_path, read_feather_gcs, batch_read_swc_from_gcs, split_neurons_by_compartment, read_obj_from_gcs

print("✓ All packages and helper functions imported")


In [ ]:
# Dataset and subset configuration
DATASET = "banc_746"
DATASET_ID = "banc_746_id"

# Tutorial 02 focuses on front leg neurons as a manageable subset
SUBSET_NAME = "front_leg"  # Anatomical subset for morphology analysis
dataset_base = "banc"       # Base dataset name

# Neuropil pattern for mesh loading (T1 leg neuropil)
NEUROPIL_PATTERN = r"T1.*leg"  # Regex pattern to match T1 leg neuropil meshes

# Data location
DATA_PATH = "gs://sjcabs_2025_data"
USE_GCS = DATA_PATH.startswith("gs://")

# Environment detection
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Image output directory
import os
IMG_DIR = "images/tutorial_02"
os.makedirs(IMG_DIR, exist_ok=True)

print(f"Dataset: {DATASET}")
print(f"Subset: {SUBSET_NAME}")
print(f"Neuropil pattern: {NEUROPIL_PATTERN}")
print(f"Data location: {DATA_PATH}")
print(f"Using GCS: {USE_GCS}")
print(f"Running in Colab: {IN_COLAB}")
print(f"Images will be saved to: {IMG_DIR}")


## Setup GCS Access

**Authentication required:** Before running with GCS, authenticate:

```bash
gcloud auth application-default login
```

In [ ]:
# Setup GCS filesystem if needed
if USE_GCS:
    gcs = gcsfs.GCSFileSystem(token='google_default')
    print("✓ GCS filesystem initialized")
else:
    gcs = None
    print("Using local filesystem")

## Setup File Paths

In [ ]:
# Construct paths
meta_path = construct_path(DATA_PATH, DATASET, "meta")
skeletons_path = construct_path(DATA_PATH, DATASET, "skeletons")

# Extract base dataset name
dataset_base = DATASET.split("_")[0]

print("File paths:")
print(f"  Metadata: {meta_path}")
print(f"  Skeletons: {skeletons_path}")

### Load Subset Data

We'll load the subset edgelist to identify which neurons are in the front leg subset, then filter the metadata accordingly.

In [ ]:
# Construct path to subset edgelist
dataset_base = DATASET.split('_')[0]
subset_dir = f"{DATA_PATH}/{dataset_base}/{SUBSET_NAME}"
edgelist_path = f"{subset_dir}/{DATASET}_{SUBSET_NAME}_simple_edgelist.feather"

print(f"Loading subset edgelist from: {edgelist_path}")

# Read the subset edgelist
edgelist = read_feather_gcs(edgelist_path, gcs_fs=gcs)

# Get unique neuron IDs from pre and post columns
subset_ids = set(edgelist['pre'].unique()) | set(edgelist['post'].unique())

print(f"Found {len(subset_ids):,} unique neurons in {SUBSET_NAME} subset")

---

## Read Meta Data

In [ ]:
# Load full metadata
meta_full = read_feather_gcs(meta_path, gcs_fs=gcs)

# Filter metadata to neurons in this subset
meta = meta_full[meta_full[DATASET_ID].isin(subset_ids)].copy()

print(f"\nLoaded {len(meta):,} neurons in {SUBSET_NAME} subset")
print(f"(From {len(meta_full):,} total proofread neurons)")
meta.head()

### Explore the Dataset

Let's examine the available neurons in this dataset:

### Examine Cell Types

What are the top cell types in this dataset?

In [ ]:
# Count by cell type
cell_type_counts = meta['cell_type'].value_counts().head(15)

print("Top 15 cell types in dataset:")
print(cell_type_counts)

# Create bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cell_type_counts.index,
    y=cell_type_counts.values,
    marker_color='steelblue',
    text=cell_type_counts.values,
    textposition='outside'
))

fig.update_layout(
    title=f"Top Cell Types: {DATASET}",
    xaxis_title="Cell Type",
    yaxis_title="Count",
    template="plotly_white",
    height=500,
    xaxis_tickangle=-45
)

fig.show()

---

## Read Neuron Skeletons (.swc files)

Neuron skeletons are stored as SWC files in the GCS bucket. We'll use navis to read them.

### Load a Single Neuron

Let's start by loading one DNg12 neuron:

In [ ]:
# Filter for DNg12 neurons from full metadata
meta_dng12 = meta[
    meta['cell_type'].str.contains('DNg12', case=False, na=False)
]

# Read ALL DNg12 neurons
if len(meta_dng12) > 0:
    # Get all DNg12 neuron IDs
    neuron_ids = meta_dng12[DATASET_ID].values
    print(f"Found {len(neuron_ids)} DNg12 neurons")
    
    # Create filenames
    filenames = [f"{nid}.swc" for nid in neuron_ids]
    
    # Read neurons
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(
            gcs,
            gcs_skeleton_path,
            filenames,
            show_progress=False
        )
    else:
        neurons = navis.NeuronList([
            navis.read_swc(f"{skeletons_path}/{fname}")
            for fname in filenames
        ])
    
    # Set neuron IDs as names for plotly hover labels
    for j, neuron_id in enumerate(neuron_ids):
        neurons[j].name = str(neuron_id)

    print(f"✓ Loaded {len(neurons)} DNg12 neurons")
else:
    print("No DNg12 neurons found")
    neurons = navis.NeuronList([])

### 3D Visualization with Plotly

navis makes it easy to visualize neurons in 3D with plotly backend:

In [ ]:
# Plot a single DNg12 neuron example
if len(neurons) > 0:
    # Select first neuron
    neuron = neurons[0]
    
    # Plot with navis (neurons are in nm, same as mesh coordinates)
    navis.plot3d(
        neuron,
        backend='plotly',
        color='darkblue',
        width=1200,
        height=800,
        title=f"DNg12 Neuron Example (1 of {len(neurons)})"
    )
else:
    print("No neurons to plot")

---

## Read Neuropil Meshes

Neuropil meshes are stored as OBJ files. We can load them and visualize neurons in their anatomical context.

In [ ]:
# Step 1: Load large region meshes (brain + VNC) from obj/ directory
region_path = f"{DATA_PATH}/{dataset_base}/obj"
print(f"Large region meshes path: {region_path}")

if USE_GCS:
    gcs_region_path = region_path.replace("gs://", "")
    region_files = gcs.ls(gcs_region_path)
    region_files = [f"gs://{f}" for f in region_files if f.endswith('.obj')]
else:
    import glob
    region_files = glob.glob(f"{region_path}/*.obj")

print(f"Found {len(region_files)} region mesh files")

# Find VNC and brain meshes
vnc_files = [f for f in region_files if 'vnc' in f.lower()]
brain_files = [f for f in region_files if 'brain' in f.lower()]

# Step 2: Load specific neuropil meshes from obj/neuropils/
neuropil_path = f"{DATA_PATH}/{dataset_base}/obj/neuropils"
print(f"Neuropil meshes path: {neuropil_path}")

if USE_GCS:
    gcs_neuropil_path = neuropil_path.replace("gs://", "")
    neuropil_files = gcs.ls(gcs_neuropil_path)
    neuropil_files = [f"gs://{f}" for f in neuropil_files if f.endswith('.obj')]
else:
    neuropil_files = glob.glob(f"{neuropil_path}/*.obj")

# Find T1 leg neuropil mesh
leg_files = [f for f in neuropil_files if re.search(NEUROPIL_PATTERN, f, re.IGNORECASE)]

# Collect meshes to load
meshes_to_load = []
mesh_labels = []

if brain_files:
    meshes_to_load.append(brain_files[0])
    mesh_labels.append('brain')

if vnc_files:
    meshes_to_load.append(vnc_files[0])
    mesh_labels.append('vnc')

if leg_files:
    meshes_to_load.append(leg_files[0])
    mesh_labels.append('leg')

print(f"Total meshes to load: {len(meshes_to_load)}")

# Store for later use
obj_files = meshes_to_load  # Keep for compatibility with next cell


### Load and Visualize Multiple Neurons with Neuropil Context

Let's load VNC and leg neuropil meshes and visualize multiple DNg12 neurons in their anatomical context, each in a different color:

In [ ]:
# Load the meshes
loaded_meshes = []
for mesh_path in meshes_to_load:
    try:
        if USE_GCS:
            mesh = read_obj_from_gcs(gcs, mesh_path.replace('gs://', ''))
        else:
            mesh = trimesh.load(mesh_path)
        loaded_meshes.append(mesh)
        print(f"✓ Loaded: {mesh_path.split('/')[-1]}")
    except Exception as e:
        print(f"✗ Failed to load {mesh_path}: {e}")

print(f"\n✓ Loaded {len(loaded_meshes)} meshes")

# Convert trimesh meshes to navis.Volume objects for visualization
if len(meta_dng12) > 0 and len(loaded_meshes) > 0:
    print(f"\nVisualizing {len(neurons)} DNg12 neurons with {len(loaded_meshes)} meshes using navis...")

    # Convert loaded_meshes to navis.Volume objects with color set directly
    # CRITICAL: Volumes don't consume color entries from plot3d()
    # Set color directly on Volume objects using RGBA tuple
    mesh_volumes = []

    for i, (mesh, label) in enumerate(zip(loaded_meshes, mesh_labels)):
        # Set alpha based on label
        if label == 'leg':
            alpha = 0.3  # Leg more visible
        else:  # brain or vnc
            alpha = 0.1  # Very transparent

        # Create navis.Volume with color set directly
        # lightgrey RGB = (0.827, 0.827, 0.827)
        volume = navis.Volume(
            mesh.vertices,
            mesh.faces,
            name=label,
            color=(0.827, 0.827, 0.827, alpha)
        )
        mesh_volumes.append(volume)

    # Create NeuronList
    neuronlist = navis.NeuronList(neurons)

    # Plot with navis using correct pattern: [volumes..., neuronlist]
    # CRITICAL: Volumes don't consume color entries from plot3d()
    # Only pass colors for NEURONS (navis unpacks NeuronList)
    plot_objects = mesh_volumes + [neuronlist]
    neuron_colors = ['red'] * len(neuronlist)  # One color per neuron
    neuron_alphas = [1.0] * len(neuronlist)   # One alpha per neuron

    fig = navis.plot3d(
        plot_objects,
        color=neuron_colors,  # ONLY neuron colors!
        alpha=neuron_alphas,   # ONLY neuron alphas!
        backend='plotly',
        width=1400,
        height=900,
        title=f'{len(neurons)} DNg12 Neurons with Neuropil Context'
    )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
    else:
        print("Note: Plot was displayed inline or could not be generated as a figure object")
else:
    print("⚠ No neurons or meshes to visualize")


---

## Co-plotting Neurons Across Datasets

One powerful feature of our data organization is that we provide neuron skeletons in both native space AND in BANC space. This enables easy co-visualization of neurons from different datasets.

Let's load EPG neurons from the **maleCNS** dataset (which are already transformed to BANC space) and co-plot them with our BANC EPG neurons:

In [ ]:
# Switch to maleCNS dataset and filter for EPG neurons
dataset_male = "malecns_09"
dataset_male_id = "malecns_09_id"

# Read maleCNS meta data
meta_male_path = construct_path(DATA_PATH, dataset_male, "meta")
meta_male_full = read_feather_gcs(meta_male_path, gcs_fs=gcs)

# Filter for EPG neurons in maleCNS
meta_male_epg = meta_male_full[
    meta_male_full['cell_type'].str.contains('EPG', case=False, na=False)
]

print(f"Found {len(meta_male_epg)} EPG neurons in maleCNS")

# Also filter BANC metadata for EPG neurons (use meta_full, not meta subset!)
meta_epg_banc = meta_full[
    meta_full['cell_type'].str.contains('EPG', case=False, na=False)
]

print(f"Found {len(meta_epg_banc)} EPG neurons in BANC")

if len(meta_male_epg) > 0 and len(meta_epg_banc) > 0:
    # Read maleCNS EPG neurons (in BANC space)
    skeletons_male_banc_path = construct_path(
        DATA_PATH,
        dataset_male,
        "skeletons",
        space_suffix="banc_space"
    )

    male_ids = meta_male_epg[dataset_male_id].values
    print(f"Reading {len(male_ids)} maleCNS EPG neurons from BANC space...")

    male_filenames = [f"{nid}.swc" for nid in male_ids]

    if USE_GCS:
        gcs_male_path = skeletons_male_banc_path.replace("gs://", "")
        epg_neurons_malecns = batch_read_swc_from_gcs(
            gcs,
            gcs_male_path,
            male_filenames,
            show_progress=False
        )
    else:
        epg_neurons_malecns = navis.NeuronList([
            navis.read_swc(f"{skeletons_male_banc_path}/{fname}")
            for fname in male_filenames
        ])

    print(f"✓ Loaded {len(epg_neurons_malecns)} maleCNS EPG neurons")

    # Read BANC EPG neurons
    banc_ids = meta_epg_banc[DATASET_ID].values
    print(f"Reading {len(banc_ids)} BANC EPG neurons...")

    banc_filenames = [f"{nid}.swc" for nid in banc_ids]

    if USE_GCS:
        gcs_banc_path = skeletons_path.replace("gs://", "")
        epg_neurons_banc = batch_read_swc_from_gcs(
            gcs,
            gcs_banc_path,
            banc_filenames,
            show_progress=False
        )
    else:
        epg_neurons_banc = navis.NeuronList([
            navis.read_swc(f"{skeletons_path}/{fname}")
            for fname in banc_filenames
        ])

    print(f"✓ Loaded {len(epg_neurons_banc)} BANC EPG neurons")

    # Co-plot BANC and maleCNS EPG neurons using navis.plot3d()
    # Prepare volumes
    mesh_volumes = []
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 0:
        for i, mesh in enumerate(loaded_meshes):
            # Set color directly on Volume (lightgrey with low alpha)
            mesh_volume = navis.Volume(
                mesh.vertices,
                mesh.faces,
                name=f'Neuropil {i+1}',
                color=(0.827, 0.827, 0.827, 0.1)  # lightgrey RGBA
            )
            mesh_volumes.append(mesh_volume)

    # Create NeuronLists (plot all neurons from each dataset)
    banc_neuronlist = navis.NeuronList(epg_neurons_banc)
    male_neuronlist = navis.NeuronList(epg_neurons_malecns)

    # Plot with navis using list pattern
    # CRITICAL: Volumes don't consume color entries from plot3d()
    # Only pass colors for neurons (navis unpacks NeuronLists)
    plot_objects = mesh_volumes + [banc_neuronlist, male_neuronlist]
    banc_colors = ['navy'] * len(banc_neuronlist)
    male_colors = ['red'] * len(male_neuronlist)
    neuron_colors = banc_colors + male_colors  # ONLY neuron colors

    # Alpha only for neurons (not volumes)
    banc_alphas = [1.0] * len(banc_neuronlist)
    male_alphas = [1.0] * len(male_neuronlist)
    neuron_alphas = banc_alphas + male_alphas

    fig = navis.plot3d(
        plot_objects,
        color=neuron_colors,  # ONLY neuron colors!
        alpha=neuron_alphas,   # ONLY neuron alphas!
        backend='plotly',
        width=1200,
        height=800,
        title="EPG Neurons: BANC (navy) vs maleCNS (red) in BANC Space"
    )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    if len(meta_male_epg) == 0:
        print("No EPG neurons found in maleCNS")
    if len(meta_epg_banc) == 0:
        print("No EPG neurons found in BANC")


---

## Load Multiple Neurons for NBLAST

Let's load neurons from the ventral nerve cord for morphological comparison. We'll filter for intrinsic VNC neurons restricted to a single leg neuromere, and take one example per cell type for computational efficiency:

In [ ]:
# Filter for intrinsic VNC neurons in single leg neuromere
meta_vnc = meta[
    (meta['super_class'] == 'ventral_nerve_cord_intrinsic') &
    (meta['cell_class'] == 'single_leg_neuromere')
].copy()

print(f"Found {len(meta_vnc)} VNC intrinsic neurons in single leg neuromere")

# Sample 100 neurons for morphometric analysis
# Note: Sampling is for tutorial performance, not usually done in real analysis
n_sample = min(100, len(meta_vnc))
meta_vnc_sampled = meta_vnc.sample(n=n_sample, random_state=42)

print(f"Sampled {len(meta_vnc_sampled)} neurons for analysis")
print(f"\nTop cell types:")
print(meta_vnc_sampled['cell_type'].value_counts().head(10))

# Get neuron IDs
sampled_ids = meta_vnc_sampled[DATASET_ID].values

if len(sampled_ids) > 0:
    print(f"\nLoading {len(sampled_ids)} neurons for NBLAST analysis...")
    
    # Prepare filenames
    swc_filenames = [f"{nid}.swc" for nid in sampled_ids]
    
    # Batch read from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, swc_filenames, show_progress=True)

        # Set neuron IDs as names for plotly hover labels
        for j, neuron_id in enumerate(sampled_ids):
            neurons[j].name = str(neuron_id)
    else:
        # Local reading
        neurons = []
        for fname in tqdm(swc_filenames):
            try:
                n = navis.read_swc(f"{skeletons_path}/{fname}")
                neurons.append(n)
            except Exception as e:
                print(f"Error reading {fname}: {e}")
        neurons = navis.NeuronList(neurons)
    
    print(f"\n✓ Loaded {len(neurons)} neurons successfully")
else:
    print("No neurons found matching the filter criteria")
    neurons = navis.NeuronList([])

### Morphometric Analysis

Calculate morphological properties for all neurons:

In [ ]:
# Convert to microns
neurons_um = neurons / 1000

print(f"\nComputing Strahler orders for {len(neurons_um)} neurons...")

# Calculate Strahler index for all neurons at once
# This adds a 'strahler_index' column to each neuron's nodes DataFrame
navis.morpho.strahler_index(neurons_um)

print("✓ Strahler indices computed!")

# Collect statistics using navis built-in methods
strahler_df = neurons_um.summary()[['name', 'n_nodes', 'cable_length']].copy()
strahler_df.columns = ['neuron_id', 'n_nodes', 'cable_length_um']
strahler_df['max_strahler_order'] = neurons_um.apply(lambda x: x.nodes.strahler_index.max())

# Add cell type information
cell_type_map = dict(zip(meta_vnc_sampled[DATASET_ID].values, 
                         meta_vnc_sampled['cell_type'].values))
strahler_df['cell_type'] = strahler_df['neuron_id'].map(cell_type_map)

print(f"\nStrahler Order Summary:")
print(f"  Neurons analyzed: {len(strahler_df)}")
print(f"  Max Strahler order range: {strahler_df['max_strahler_order'].min():.0f} - {strahler_df['max_strahler_order'].max():.0f}")
print(f"  Mean max Strahler order: {strahler_df['max_strahler_order'].mean():.1f} ± {strahler_df['max_strahler_order'].std():.1f}")

strahler_df.head(10)

### Strahler Order Analysis**Strahler order** is a numerical measure of branching complexity that classifies branches hierarchically:<p align="center">  <img src="https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/inst/images/strahler_order.png" alt="Strahler Order Diagram" width="70%"></p>**Classification rules:**- **Order 1**: Terminal branches (no children)- **Order n+1**: Parent of two order-n branches- **Higher order**: Parent takes the maximum order of its children when orders differThis metric reveals:- **Overall complexity**: Maximum Strahler order indicates branching hierarchy depth- **Cable distribution**: How much cable is allocated to different branching levels- **Arbor architecture**: Balance between terminal branches (order 1) and main trunks (high orders)Let's visualize the Strahler order distribution across our VNC neurons:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette("Set2")

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Distribution of maximum Strahler orders
ax1 = axes[0, 0]
sns.histplot(
    data=strahler_df,
    x='max_strahler_order',
    bins=range(int(strahler_df['max_strahler_order'].min()), 
               int(strahler_df['max_strahler_order'].max()) + 2),
    ax=ax1,
    kde=False
)
ax1.set_xlabel('Maximum Strahler Order', fontsize=12)
ax1.set_ylabel('Number of Neurons', fontsize=12)
ax1.set_title('Distribution of Maximum Strahler Orders', fontsize=13, fontweight='bold')

# 2. Strahler order vs cable length
ax2 = axes[0, 1]
sns.scatterplot(
    data=strahler_df,
    x='max_strahler_order',
    y='cable_length_um',
    ax=ax2,
    alpha=0.6
)
ax2.set_xlabel('Maximum Strahler Order', fontsize=12)
ax2.set_ylabel('Cable Length (µm)', fontsize=12)
ax2.set_title('Strahler Order vs Cable Length', fontsize=13, fontweight='bold')

# 3. Strahler order vs number of nodes
ax3 = axes[1, 0]
sns.scatterplot(
    data=strahler_df,
    x='max_strahler_order',
    y='n_nodes',
    ax=ax3,
    alpha=0.6,
    color='coral'
)
ax3.set_xlabel('Maximum Strahler Order', fontsize=12)
ax3.set_ylabel('Number of Nodes', fontsize=12)
ax3.set_title('Strahler Order vs Node Count', fontsize=13, fontweight='bold')

# 4. Box plot of Strahler orders
ax4 = axes[1, 1]
sns.boxplot(
    data=strahler_df,
    x='max_strahler_order',
    ax=ax4
)
ax4.set_xlabel('Maximum Strahler Order', fontsize=12)
ax4.set_ylabel('', fontsize=12)
ax4.set_title('Strahler Order Variability', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("Strahler Order Analysis Summary")
print("="*60)
print(f"\nNeurons by maximum Strahler order:")
order_counts = strahler_df['max_strahler_order'].value_counts().sort_index()
for order, count in order_counts.items():
    pct = (count / len(strahler_df)) * 100
    print(f"  Order {int(order)}: {count} neurons ({pct:.1f}%)")

print(f"\nMorphological complexity:")
print(f"  Mean max order: {strahler_df['max_strahler_order'].mean():.2f}")
print(f"  Std dev: {strahler_df['max_strahler_order'].std():.2f}")
print(f"  Range: {int(strahler_df['max_strahler_order'].min())} - {int(strahler_df['max_strahler_order'].max())}")

---

## NBLAST Morphological Similarity

**NBLAST** (Neuron BLAST) is an algorithm for comparing neuron morphology based on local geometry. It works by:

1. Converting neurons to point clouds with tangent vectors (dotprops)
2. For each point in query neuron, finding nearest point in target
3. Computing similarity based on distance and angle between tangent vectors

navis includes a fast NBLAST implementation (uses compiled Rust code under the hood).

**Note:** For this tutorial, we'll demonstrate the code but not execute it since it requires additional computation time. Remove `eval=False` to run it yourself.

In [ ]:
# Convert neurons to dotprops (point clouds with tangent vectors)
# This removes connectivity but preserves spatial structure
neurons_dp = navis.make_dotprops(neurons_um, k=5)

print(f"  {len(neurons_dp)} neurons ready for NBLAST")

# Run NBLAST all-by-all comparison
# Use nblast_allbyall for better performance
# Use all available cores (minus 1 to keep system responsive)
import multiprocessing
n_cores = max(1, multiprocessing.cpu_count() - 1)

print(f"\nRunning NBLAST using {n_cores} cores (this may take a moment)...")
nblast_scores = navis.nblast_allbyall(neurons_dp, n_cores=n_cores)
# Note: Scores are already normalized (self-hit = 1)

print(f"Score matrix shape: {nblast_scores.shape}")

# Display score matrix
nblast_scores.head()

### Hierarchical Clustering

Use NBLAST scores to cluster neurons by morphological similarity:

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, cut_treefrom scipy.spatial.distance import squareformimport plotly.figure_factory as ff# Convert NBLAST scores to distance matrix# NBLAST scores are already normalized (self-hit = 1) and represent similarity# Convert similarity to distance for clusteringdistance_matrix = 1 - nblast_scores.values# Convert to condensed distance matrix for linkage# Only use upper triangle (distance matrix is symmetric)condensed_dist = squareform(distance_matrix, checks=False)# Perform hierarchical clusteringlinkage_matrix = linkage(condensed_dist, method='ward')# Choose number of clustersk = 8print(f"Cutting dendrogram into {k} clusters...")# Cut tree to get cluster assignmentsclusters = cut_tree(linkage_matrix, n_clusters=k).flatten() + 1  # +1 to start from 1# Get cut height for k clusters# Height at which we cut is at the (n-k)th mergecut_height = linkage_matrix[len(linkage_matrix) - k, 2]print(f"Cut height for k={k}: {cut_height:.4f}")print(f"\nCluster sizes:")cluster_counts = pd.Series(clusters).value_counts().sort_index()print(cluster_counts)# Create dendrogram with plotly# First create using scipy to get dendrogram coordinatesdend = dendrogram(linkage_matrix, no_plot=True)# Get leaf order and corresponding clustersleaf_order = dend['leaves']leaf_clusters = clusters[leaf_order]# Get cell types for leaves (map back through leaf_order to original neurons)leaf_cell_types = strahler_df['cell_type'].iloc[leaf_order].values# Create color map for clustersimport plotly.express as pxcolors_list = px.colors.qualitative.Light24 + px.colors.qualitative.Dark24cluster_colors = {i: colors_list[i % len(colors_list)] for i in range(1, k+1)}leaf_colors = [cluster_colors[c] for c in leaf_clusters]# Create figurefig = go.Figure()# Add dendrogram segmentsfor i, (x, y) in enumerate(zip(dend['icoord'], dend['dcoord'])):    fig.add_trace(go.Scatter(        x=x,        y=y,        mode='lines',        line=dict(color='grey', width=1),        hoverinfo='skip',        showlegend=False    ))# Add colored points at leaves with cell_type hover textleaf_x = [(dend['icoord'][i][1] + dend['icoord'][i][2]) / 2           for i in range(len(dend['icoord']))           if dend['dcoord'][i][0] == 0]leaf_y = [0] * len(leaf_x)# Group leaves by cluster for legend, including cell_type in hover textfor cluster_id in sorted(cluster_counts.index):    cluster_mask = leaf_clusters == cluster_id    cluster_x = [x for i, x in enumerate(leaf_x) if cluster_mask[i]]    cluster_y = [0] * len(cluster_x)        # Get cell types for this cluster's leaves    cluster_cell_types = [leaf_cell_types[i] for i, mask in enumerate(cluster_mask) if mask]        fig.add_trace(go.Scatter(        x=cluster_x,        y=cluster_y,        mode='markers',        marker=dict(size=8, color=cluster_colors[cluster_id]),        name=f'Cluster {cluster_id}',        hovertext=[f'Cell Type: {ct}<br>Cluster: {cluster_id}' for ct in cluster_cell_types],        hoverinfo='text'    ))# Add horizontal cut linefig.add_hline(    y=cut_height,    line=dict(color='red', width=2, dash='dash'),    annotation_text=f'Cut height (k={k})',    annotation_position='right')# Update layoutfig.update_layout(    title=dict(        text=f'Hierarchical Clustering of Neuron Morphology (NBLAST)<br><sub>{DATASET} - {SUBSET_NAME} (k={k} clusters)</sub>',        x=0.5,        xanchor='center'    ),    xaxis=dict(        title='Neurons',        showticklabels=False,        showgrid=False    ),    yaxis=dict(        title='Height (Ward Distance)',        showgrid=True,        gridcolor='lightgrey'    ),    plot_bgcolor='white',    width=1200,    height=600,    hovermode='closest',    legend=dict(        orientation='v',        yanchor='top',        y=1,        xanchor='left',        x=1.01,        bgcolor='rgba(255, 255, 255, 0.8)',        bordercolor='grey',        borderwidth=1    ))# Save static version# Save static version (requires kaleido)try:    fig.write_image(f"{IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png",                    width=1400, height=700, scale=2)    print(f"\n✓ Dendrogram saved to {IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png")except Exception as e:    print(f"\nℹ Could not save static image: {e}")    print("  To enable image export, install kaleido: pip install kaleido")# Display interactive versionfig.show()

### Interactive Cluster Visualization

Now let's create a single interactive 3D plot showing all clusters together. Each cluster is a separate trace that can be toggled on/off by clicking its legend entry:

In [ ]:
# Create unified interactive plot with all clusters
# Convert leg mesh to navis.Volume for proper visualization
leg_volume = None
if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
    leg_mesh = loaded_meshes[2]  # Third mesh is leg neuropil
    leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

# Collect cluster neurons (in nm, not µm!)
all_cluster_neurons = []
all_cluster_colors = []

for cluster_id in sorted(cluster_counts.index):
    # Get neurons in this cluster (using neurons in nm, not neurons_um!)
    cluster_mask = clusters == cluster_id
    cluster_neurons_list = [neurons[i] for i in range(len(neurons)) if cluster_mask[i]]

    if len(cluster_neurons_list) > 0:
        all_cluster_neurons.extend(cluster_neurons_list)
        all_cluster_colors.extend([cluster_colors[cluster_id]] * len(cluster_neurons_list))

# Create NeuronList for all cluster neurons
all_cluster_neuronlist = navis.NeuronList(all_cluster_neurons)

if len(all_cluster_neuronlist) > 0:
    print(f"\nPlotting {len(all_cluster_neuronlist)} neurons in {k} clusters with neuropil context...")

    # Plot with navis using list pattern
    if leg_volume is not None:
        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_alphas = [1.0] * len(all_cluster_neuronlist)

        fig = navis.plot3d(
            [leg_volume, all_cluster_neuronlist],
            color=all_cluster_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1400,
            height=900,
            title=f'NBLAST Clusters: All {k} Clusters (Neurons in nm)'
        )
    else:
        fig = navis.plot3d(
            all_cluster_neuronlist,
            color=all_cluster_colors,
            backend='plotly',
            width=1400,
            height=900,
            title=f'NBLAST Clusters: All {k} Clusters (Neurons in nm)'
        )

    if fig is not None:
        # Update layout for nm coordinates
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            ),
            showlegend=False
        )
        fig.show()
else:
    print("No cluster neurons to plot")


### Visualize Individual Clusters

Let's visualize each cluster separately to examine morphological patterns within clusters.

In [ ]:
def visualize_cluster(cluster_num, neurons, clusters, cluster_colors):    """    Visualize a single cluster of neurons.        Parameters    ----------    cluster_num : int        The cluster number to visualize    neurons : list        List of all neurons    clusters : array        Array of cluster assignments for each neuron    cluster_colors : dict        Dictionary mapping cluster numbers to colors    """    # Get neurons for this cluster    cluster_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == cluster_num]        if len(cluster_neurons) == 0:        print(f'No neurons in cluster {cluster_num}')        return        print(f'Cluster {cluster_num}: {len(cluster_neurons)} neurons')        # Prepare volume if available    if 'loaded_meshes' in globals() and len(loaded_meshes) > 2:        leg_mesh = loaded_meshes[2]        leg_volume = navis.Volume(            leg_mesh.vertices,             leg_mesh.faces,             name='Leg Neuropil',             color=(0.827, 0.827, 0.827, 0.3)        )                # Create NeuronList and plot with volume        cluster_neuronlist = navis.NeuronList(cluster_neurons)                # CRITICAL: Volumes don't consume colors from plot3d()        # Only pass neuron colors (navis unpacks NeuronList)        neuron_colors = [cluster_colors[cluster_num]] * len(cluster_neuronlist)        neuron_alphas = [1.0] * len(cluster_neuronlist)                fig = navis.plot3d(            [leg_volume, cluster_neuronlist],            color=neuron_colors,  # ONLY neuron colors!            alpha=neuron_alphas,            backend='plotly',            width=1000,            height=800,            title=f'Cluster {cluster_num} Morphology ({len(cluster_neurons)} neurons)'        )    else:        # Plot neurons only if no volume available        cluster_neuronlist = navis.NeuronList(cluster_neurons)                fig = navis.plot3d(            cluster_neuronlist,            color=cluster_colors[cluster_num],            backend='plotly',            width=1000,            height=800,            title=f'Cluster {cluster_num} Morphology ({len(cluster_neurons)} neurons)'        )        if fig is not None:        fig.update_layout(            scene=dict(                xaxis_title='X (nm)',                yaxis_title='Y (nm)',                zaxis_title='Z (nm)',                aspectmode='data'            )        )        fig.show()

In [ ]:
# Visualize all clusters using the functionfor cluster_num in range(1, k+1):    visualize_cluster(cluster_num, neurons, clusters, cluster_colors)

## Your Turn: New Subset

Now try this analysis yourself with a different dataset!

**Exercise:** Switch the pre-prepared subset to `antennal_lobe`

To work with a different subset, change the subset configuration at the top:
```python
SUBSET_NAME = "antennal_lobe"
NEUROPIL_PATTERN = "AL"
```

Then re-run the entire notebook to see how the results differ!

# Extensions

In the extended code blocks, you can learn how to 1. transform data yourself between template spaces, and 2. visualise data split by axon and dendrite.

---

## Extension 1: Template Brain Transformations

While we provide pre-transformed skeletons, you can also perform transformations yourself using **flybrains** and **navis**.

The [**flybrains**](https://github.com/navis-org/navis-flybrains) package provides template brain transformations for 31+ Drosophila brain spaces including FAFB14, FLYWIRE, JRC2018F/M/U, JRCFIB2022M (maleCNS), FANC, BANC, and MANC.

This is useful when:
- Working with custom neurons not in our dataset
- Transforming synapses or other 3D data
- Needing intermediate template spaces

**Note:** This requires flybrains to be installed. We installed it earlier in this tutorial.

In [ ]:
# Check if flybrains is installed
try:
    import flybrains
    print(f"✓ flybrains version: {flybrains.__version__}")
    FLYBRAINS_AVAILABLE = True
except ImportError:
    print("ℹ flybrains not installed (optional)")
    print("  flybrains enables spatial transformations between datasets")
    print("  For this tutorial, we use pre-transformed skeletons instead")
    print()
    print("  To install: pip install flybrains")
    print("  Documentation: https://github.com/navis-org/navis-flybrains")
    FLYBRAINS_AVAILABLE = False

if FLYBRAINS_AVAILABLE:
    print()
    print("=" * 70)
    print("Flybrains Transform Example")
    print("=" * 70)
    print()
    print("✓ flybrains is installed and ready")
    print()
    print("Example transforms available:")
    print("  • MANC → JRCVNC2018M → BANC")
    print("  • FAFB → JRC2018F → BANC")
    print("  • maleCNS (JRCFIB2022M) → BANC")
    print()
    print("For detailed transform examples, see:")
    print("  https://navis-org.github.io/navis/generated/gallery/6_misc/tutorial_misc_01_transforms/")
    print()
    print("Note: For this tutorial, we use pre-transformed skeletons which are")
    print("      faster and more convenient for bulk analysis.")


Now let's use these transforms to move data from MANC space into BANC space. Again we'll pick DNg12 neurons.

In [ ]:
# Extension 1: Cross-Dataset Co-Visualization
#
# This section demonstrates co-plotting neurons from multiple datasets in BANC space.
# All datasets provide pre-transformed skeletons in BANC space for easy comparison.

# Check if flybrains is available (optional for manual transforms)
try:
    import flybrains
    print(f"✓ flybrains version: {flybrains.__version__}")
    FLYBRAINS_AVAILABLE = True
except ImportError:
    print("ℹ flybrains not installed (optional for manual transforms)")
    print("  Install with: pip install flybrains")
    print("  For this tutorial, we'll use pre-transformed skeletons instead")
    FLYBRAINS_AVAILABLE = False

print("\n" + "="*70)
print("Cross-Dataset Comparison: DNg12 Neurons")
print("="*70)

# Initialize variables to ensure they exist
manc_dng12_banc = navis.NeuronList([])
neurons_male_banc = navis.NeuronList([])

# Load MANC DNg12 neurons (in BANC space)
dataset_manc = "manc_121"
dataset_manc_id = "manc_121_id"

# Read MANC meta data
meta_manc_path = construct_path(DATA_PATH, dataset_manc, "meta")
meta_manc_full = read_feather_gcs(meta_manc_path, gcs_fs=gcs)

# Filter for DNg12 neurons
meta_manc_dng12 = meta_manc_full[
    meta_manc_full['cell_type'].str.contains('DNg12', case=False, na=False)
]
print(f"\nFound {len(meta_manc_dng12)} DNg12 neurons in MANC")

if len(meta_manc_dng12) > 0:
    # Load MANC neurons in BANC space (pre-transformed)
    skeletons_manc_banc_path = construct_path(
        DATA_PATH,
        dataset_manc,
        "skeletons",
        space_suffix="banc_space"
    )
    
    manc_ids = meta_manc_dng12[dataset_manc_id].values[:5]  # Limit to 5 for speed
    manc_filenames = [f"{nid}.swc" for nid in manc_ids]
    
    print(f"Loading {len(manc_ids)} MANC DNg12 neurons (in BANC space)...")
    
    if USE_GCS:
        gcs_manc_path = skeletons_manc_banc_path.replace("gs://", "")
        manc_dng12_banc = batch_read_swc_from_gcs(
            gcs,
            gcs_manc_path,
            manc_filenames,
            show_progress=False
        )
    else:
        manc_dng12_banc = navis.NeuronList([
            navis.read_swc(f"{skeletons_manc_banc_path}/{fname}")
            for fname in manc_filenames
        ])
    
    print(f"✓ Loaded {len(manc_dng12_banc)} MANC neurons")

# Load maleCNS DNg12 neurons for comparison (independent of MANC)
dataset_male = "malecns_09"
dataset_male_id = "malecns_09_id"

# Read maleCNS meta data
meta_male_path = construct_path(DATA_PATH, dataset_male, "meta")
meta_male_full = read_feather_gcs(meta_male_path, gcs_fs=gcs)

# Filter for DNg12 neurons in maleCNS
meta_male_dng12 = meta_male_full[
    meta_male_full['cell_type'].str.contains('DNg12', case=False, na=False)
]
print(f"\nFound {len(meta_male_dng12)} DNg12 neurons in maleCNS")

if len(meta_male_dng12) > 0:
    # Load maleCNS DNg12 neurons in BANC space
    skeletons_male_banc_path = construct_path(
        DATA_PATH,
        dataset_male,
        "skeletons",
        space_suffix="banc_space"
    )
    
    male_ids = meta_male_dng12[dataset_male_id].values[:5]
    male_filenames = [f"{nid}.swc" for nid in male_ids]
    
    print(f"Loading {len(male_ids)} maleCNS DNg12 neurons (in BANC space)...")
    
    if USE_GCS:
        gcs_male_path = skeletons_male_banc_path.replace("gs://", "")
        neurons_male_banc = batch_read_swc_from_gcs(
            gcs,
            gcs_male_path,
            male_filenames,
            show_progress=False
        )
    else:
        neurons_male_banc = navis.NeuronList([
            navis.read_swc(f"{skeletons_male_banc_path}/{fname}")
            for fname in male_filenames
        ])
    
    print(f"✓ Loaded {len(neurons_male_banc)} maleCNS DNg12 neurons")

# Co-visualize BANC, maleCNS, and MANC neurons (if all are available)
if 'neurons' in locals():
    if len(neurons) > 0 and len(neurons_male_banc) > 0 and len(manc_dng12_banc) > 0:
        print("\nCo-plotting DNg12 neurons from three datasets...")
        print("  BANC (navy) | maleCNS (red) | MANC (green)")
        print("  All in BANC space for direct comparison")
        
        # Limit to 3 neurons per dataset for clarity
        banc_subset = neurons[:3]
        male_subset = neurons_male_banc[:3]
        manc_subset = manc_dng12_banc[:3]
        
        # Combine neurons and colors
        # IMPORTANT: navis.plot3d unpacks NeuronLists, so we need one color per neuron
        all_neurons = banc_subset + male_subset + manc_subset
        all_colors = (['navy'] * len(banc_subset) + 
                     ['red'] * len(male_subset) + 
                     ['green'] * len(manc_subset))
        
        # Plot all together
        fig = navis.plot3d(
            all_neurons,
            color=all_colors,
            backend='plotly',
            width=1400,
            height=900,
            title='DNg12 Neurons: BANC (navy) vs maleCNS (red) vs MANC (green)'
        )
        
        if fig is not None:
            fig.update_layout(
                scene=dict(
                    xaxis_title='X (nm)',
                    yaxis_title='Y (nm)',
                    zaxis_title='Z (nm)',
                    aspectmode='data'
                )
            )
            fig.show()
        
        print("\n✓ Cross-dataset visualization complete")
        print("\nKey Points:")
        print("  • All neurons are in BANC space (pre-transformed)")
        print("  • This enables direct spatial comparison")
        print("  • No manual transform required - just load and plot!")
    else:
        print("\n⚠ Cannot co-visualize - need neurons from all three datasets")
        if 'neurons' not in locals() or len(neurons) == 0:
            print("  • Missing BANC DNg12 neurons (run cell 19)")
        if len(neurons_male_banc) == 0:
            print("  • Missing maleCNS DNg12 neurons")
        if len(manc_dng12_banc) == 0:
            print("  • Missing MANC DNg12 neurons")
else:
    print("\n⚠ Need BANC DNg12 neurons from earlier cells")
    print("  Run cell 19 to load 'neurons' variable")

print("\n" + "="*70)
print("Transformation Notes")
print("="*70)
print("\n✓ RECOMMENDED: Use pre-transformed skeletons")
print("  - Available for all datasets with space_suffix='banc_space'")
print("  - Reliable, fast, no additional setup required")
print("\nℹ Advanced: Manual transforms (requires flybrains)")
print("  - MANC → JRCVNC2018M: Available via flybrains")
print("  - JRCVNC2018M → BANC: Requires elastix (complex setup)")
print("  - For details: https://navis-org.github.io/navis/generated/gallery/6_misc/tutorial_misc_01_transforms/")


## Extension 2: Axon-Dendrite Splits

For many of our datasets (but notably not BANC yet), we have axon-dendrite splits that have been pre-calculated using a graph-theoretic algorithm from [Schneider-Mizell et al. 2016](https://elifesciences.org/articles/12059).

The method determines axon versus dendrite by "cutting" the neuron at its major information bottleneck based on synapse distributions, observing which half has more inputs versus outputs.

**Data locations:**
- The `Label` column in `.swc` skeleton files gives compartment assignments (values: "axon", "dendrite", "primary_dendrite", "primary_neurite")
- The `pre_label` and `post_label` columns in synapse tables give compartment assignments

Let's visualize some olfactory projection neurons from **FAFB** with compartments color-coded:

- **Cyan** = dendrite
- **Green** = primary dendrite (linker)
- **Orange** = axon  
- **Purple** = primary neurite tract (cell body fiber tract)

We'll plot V_l2PN projections from the V glomerulus, which sense CO2.

In [ ]:
# Switch to FAFB dataset (has compartment labels)
dataset_fafb = "fafb_783"
dataset_fafb_id = "fafb_783_id"

# Read FAFB meta data
meta_fafb_path = construct_path(DATA_PATH, dataset_fafb, "meta")
meta_fafb = read_feather_gcs(meta_fafb_path, gcs_fs=gcs)

# Filter for V_l2PN neurons
vpn_meta = meta_fafb[
    meta_fafb['cell_type'].str.contains('V_l2PN', case=False, na=False)
]
print(f"Found {len(vpn_meta)} V_l2PN neurons in FAFB")

if len(vpn_meta) > 0:
    # Get neuron IDs (limit to first 3 for visualization)
    vpn_ids = vpn_meta[dataset_fafb_id].values[:min(3, len(vpn_meta))]

    # Construct path to FAFB skeletons in BANC space (for co-visualization)
    skeletons_fafb_path = construct_path(
        DATA_PATH,
        "fafb",
        "skeletons",
        space_suffix="fafb_space"
    )

    print(f"Loading {len(vpn_ids)} FAFB V_l2PN neurons...")

    # Read FAFB neurons
    vpn_filenames = [f"{nid}.swc" for nid in vpn_ids]

    if USE_GCS:
        gcs_fafb_path = skeletons_fafb_path.replace("gs://", "")
        vpn_neurons = batch_read_swc_from_gcs(
            gcs,
            gcs_fafb_path,
            vpn_filenames,
            show_progress=False
        )
    else:
        vpn_neurons = navis.NeuronList([
            navis.read_swc(f"{skeletons_fafb_path}/{fname}")
            for fname in vpn_filenames
        ])

    print(f"✓ Loaded {len(vpn_neurons)} V_l2PN neurons")

    if len(vpn_neurons) > 0:
        # Split neurons by compartment (similar to R's hemibrainr functions)
        # This uses the 'Label' column from SWC files
        compartments = split_neurons_by_compartment(vpn_neurons)

        axon_list = compartments['axon']
        dendrite_list = compartments['dendrite']
        linker_list = compartments['linker']
        neurite_list = compartments['neurite']

        print(f"\nCompartments found:")
        print(f"  Axons: {len(axon_list)}")
        print(f"  Dendrites: {len(dendrite_list)}")
        print(f"  Linkers (primary dendrite): {len(linker_list)}")
        print(f"  Neurites (primary neurite): {len(neurite_list)}")

        # Try to load antennal lobe mesh for context
        try:
            al_mesh_path = construct_path(
                DATA_PATH,
                "fafb",
                "neuropil_meshes",
                space_suffix="fafb_space"
            )

            al_mesh_file = "AL_R.obj"

            if USE_GCS:
                gcs_al_path = al_mesh_path.replace("gs://", "") + "/" + al_mesh_file
                al_mesh = read_obj_from_gcs(gcs, gcs_al_path)
            else:
                al_mesh = trimesh.load_mesh(f"{al_mesh_path}/{al_mesh_file}")

            # Convert trimesh to navis Volume
            al_volume = navis.Volume(
                vertices=al_mesh.vertices,
                faces=al_mesh.faces,
                name='Antennal Lobe',
                color=(0.8, 0.8, 0.8, 0.1)  # Light grey, transparent
            )
            print("✓ Loaded antennal lobe mesh")
        except Exception as e:
            print(f"  ⚠ Could not load antennal lobe mesh: {e}")
            al_volume = None

        # Plot compartments with different colors (similar to R code)
        # This approach matches hemibrainr::axonic_cable(), dendritic_cable(), etc.
        print("\nVisualizing V_l2PN neurons by compartment...")

        # Build plot objects and colors manually
        plot_objects = []
        plot_colors = []

        # Add antennal lobe volume if available
        if al_volume is not None:
            plot_objects.append(al_volume)
            # Volume color is set on the object itself

        # Add compartments in order (matching R code)
        # dendrite = cyan
        if len(dendrite_list) > 0:
            for n in dendrite_list:
                plot_objects.append(n)
                plot_colors.append('cyan')

        # linker (primary dendrite) = green
        if len(linker_list) > 0:
            for n in linker_list:
                plot_objects.append(n)
                plot_colors.append('green')

        # axon = orange
        if len(axon_list) > 0:
            for n in axon_list:
                plot_objects.append(n)
                plot_colors.append('orange')

        # neurite (primary neurite) = purple
        if len(neurite_list) > 0:
            for n in neurite_list:
                plot_objects.append(n)
                plot_colors.append('purple')

        # Plot with navis.plot3d
        if len(plot_objects) > 0:
            # Adjust colors list to account for volume (if present)
            if al_volume is not None:
                # Volume doesn't consume a color from the list
                # navis.plot3d will use the color set on the Volume object
                fig = navis.plot3d(
                    plot_objects,
                    color=['lightgrey'] + plot_colors,  # First color for volume, rest for neurons
                    backend='plotly',
                    width=1200,
                    height=800,
                    title='FAFB V_l2PN Neurons - Colored by Compartment (in Antennal Lobe)'
                )
            else:
                fig = navis.plot3d(
                    plot_objects,
                    color=plot_colors,
                    backend='plotly',
                    width=1200,
                    height=800,
                    title='FAFB V_l2PN Neurons - Colored by Compartment'
                )

            if fig is not None:
                fig.show()

            print("\n✓ Compartment visualization complete")
        else:
            print("\n⚠ No compartments to plot")
    else:
        print("⚠ No V_l2PN neurons loaded")
else:
    print("⚠ No V_l2PN neurons found in FAFB - skipping compartment visualization")


### Visualize Synapses by Compartment

We can also visualize the synapses on these neurons, colored by compartment and direction:

- **Navy** = input synapse, dendrite
- **Cyan** = input synapse, axon
- **Pink** = output synapse, dendrite
- **Red** = output synapse, axon

In [ ]:
# Only run if vpn_ids exists (from previous section)
if 'vpn_ids' in locals() and len(vpn_ids) > 0:
    # Construct path to FAFB synapses file
    dataset = "fafb_783"
    synapses_path = f"{DATA_PATH}/fafb/antennal_lobe/{dataset}_antennal_lobe_synapses.feather"
    
    print("Loading FAFB antennal lobe synapses...")
    
    # Read synapses and filter for V_l2PN neurons
    vpn_synapses = read_feather_gcs(synapses_path, gcs_fs=gcs)
    vpn_synapses = vpn_synapses[
        vpn_synapses['pre'].isin(vpn_ids) | vpn_synapses['post'].isin(vpn_ids)
    ]
    
    print(f"Found {len(vpn_synapses):,} synapses for V_l2PN neurons")
    
    if len(vpn_synapses) > 0:
        # Create synapse classifications by compartment and direction
        # Output synapses on axons
        vpn_synapses_output_axons = vpn_synapses[
            vpn_synapses['pre'].isin(vpn_ids) & (vpn_synapses['pre_label'] == 'axon')
        ][['x', 'y', 'z']].values
        
        # Output synapses on dendrites
        vpn_synapses_output_dendrites = vpn_synapses[
            vpn_synapses['pre'].isin(vpn_ids) & (vpn_synapses['pre_label'] == 'dendrite')
        ][['x', 'y', 'z']].values
        
        # Input synapses on axons
        vpn_synapses_input_axons = vpn_synapses[
            vpn_synapses['post'].isin(vpn_ids) & (vpn_synapses['post_label'] == 'axon')
        ][['x', 'y', 'z']].values
        
        # Input synapses on dendrites
        vpn_synapses_input_dendrites = vpn_synapses[
            vpn_synapses['post'].isin(vpn_ids) & (vpn_synapses['post_label'] == 'dendrite')
        ][['x', 'y', 'z']].values
        
        print(f"\nSynapse counts by type:")
        print(f"  Output, axon: {len(vpn_synapses_output_axons):,}")
        print(f"  Output, dendrite: {len(vpn_synapses_output_dendrites):,}")
        print(f"  Input, axon: {len(vpn_synapses_input_axons):,}")
        print(f"  Input, dendrite: {len(vpn_synapses_input_dendrites):,}")
        
        # Verify coordinate ranges
        print(f"\nSynapse coordinate ranges:")
        all_syn_coords = np.vstack([vpn_synapses_output_axons, vpn_synapses_output_dendrites,
                                     vpn_synapses_input_axons, vpn_synapses_input_dendrites])
        print(f"  X: {all_syn_coords[:, 0].min():.0f} to {all_syn_coords[:, 0].max():.0f}")
        print(f"  Y: {all_syn_coords[:, 1].min():.0f} to {all_syn_coords[:, 1].max():.0f}")
        print(f"  Z: {all_syn_coords[:, 2].min():.0f} to {all_syn_coords[:, 2].max():.0f}")
        
        # Plot neurons with synapses
        if 'vpn_neurons' in locals() and len(vpn_neurons) > 0:
            print("\nVisualizing synapses by compartment...")
            
            # Split neurons by compartment (same as in cell 28)
            compartments = split_neurons_by_compartment(vpn_neurons)
            
            axon_list = compartments['axon']
            dendrite_list = compartments['dendrite']
            linker_list = compartments['linker']
            neurite_list = compartments['neurite']
            
            # Build plot objects and colors for neurons
            plot_objects = []
            plot_colors = []
            
            # Add compartments in order
            if len(dendrite_list) > 0:
                for n in dendrite_list:
                    plot_objects.append(n)
                    plot_colors.append('cyan')
            
            if len(linker_list) > 0:
                for n in linker_list:
                    plot_objects.append(n)
                    plot_colors.append('green')
            
            if len(axon_list) > 0:
                for n in axon_list:
                    plot_objects.append(n)
                    plot_colors.append('orange')
            
            if len(neurite_list) > 0:
                for n in neurite_list:
                    plot_objects.append(n)
                    plot_colors.append('purple')
            
            # Plot neurons by compartment first
            if len(plot_objects) > 0:
                print("\nPlotting neurons...")
                fig = navis.plot3d(
                    plot_objects,
                    color=plot_colors,
                    backend='plotly',
                    inline=False,
                    width=1400,
                    height=900,
                    title='V_l2PN Synapses by Compartment and Direction'
                )
                
                if fig is not None:
                    print("Adding synapse scatter points...")
                    # Add synapse points with small markers (size 5)
                    # to ensure visibility in nm-scale coordinates
                    
                    # Navy = input synapse, dendrite
                    if len(vpn_synapses_input_dendrites) > 0:
                        print(f"  Adding {len(vpn_synapses_input_dendrites)} input dendrite synapses (navy)...")
                        fig.add_trace(go.Scatter3d(
                            x=vpn_synapses_input_dendrites[:, 0],
                            y=vpn_synapses_input_dendrites[:, 1],
                            z=vpn_synapses_input_dendrites[:, 2],
                            mode='markers',
                            marker=dict(size=5, color='navy', opacity=0.9),
                            name='Input, dendrite',
                            showlegend=True,
                            hovertemplate='Input dendrite<br>X: %{x}<br>Y: %{y}<br>Z: %{z}<extra></extra>'
                        ))
                    
                    # Cyan = input synapse, axon
                    if len(vpn_synapses_input_axons) > 0:
                        print(f"  Adding {len(vpn_synapses_input_axons)} input axon synapses (cyan)...")
                        fig.add_trace(go.Scatter3d(
                            x=vpn_synapses_input_axons[:, 0],
                            y=vpn_synapses_input_axons[:, 1],
                            z=vpn_synapses_input_axons[:, 2],
                            mode='markers',
                            marker=dict(size=5, color='cyan', opacity=0.9),
                            name='Input, axon',
                            showlegend=True,
                            hovertemplate='Input axon<br>X: %{x}<br>Y: %{y}<br>Z: %{z}<extra></extra>'
                        ))
                    
                    # Pink = output synapse, dendrite
                    if len(vpn_synapses_output_dendrites) > 0:
                        print(f"  Adding {len(vpn_synapses_output_dendrites)} output dendrite synapses (pink)...")
                        fig.add_trace(go.Scatter3d(
                            x=vpn_synapses_output_dendrites[:, 0],
                            y=vpn_synapses_output_dendrites[:, 1],
                            z=vpn_synapses_output_dendrites[:, 2],
                            mode='markers',
                            marker=dict(size=5, color='pink', opacity=0.9),
                            name='Output, dendrite',
                            showlegend=True,
                            hovertemplate='Output dendrite<br>X: %{x}<br>Y: %{y}<br>Z: %{z}<extra></extra>'
                        ))
                    
                    # Red = output synapse, axon
                    if len(vpn_synapses_output_axons) > 0:
                        print(f"  Adding {len(vpn_synapses_output_axons)} output axon synapses (red)...")
                        fig.add_trace(go.Scatter3d(
                            x=vpn_synapses_output_axons[:, 0],
                            y=vpn_synapses_output_axons[:, 1],
                            z=vpn_synapses_output_axons[:, 2],
                            mode='markers',
                            marker=dict(size=5, color='red', opacity=0.9),
                            name='Output, axon',
                            showlegend=True,
                            hovertemplate='Output axon<br>X: %{x}<br>Y: %{y}<br>Z: %{z}<extra></extra>'
                        ))
                    
                    # Update layout
                    fig.update_layout(
                        scene=dict(
                            xaxis_title='X (nm)',
                            yaxis_title='Y (nm)',
                            zaxis_title='Z (nm)',
                            aspectmode='data'
                        )
                    )
                    
                    fig.show()
                    print("\n✓ Synapse visualization complete")
                    print(f"\nNote: Synapse marker size = 5 (small for cleaner visualization in nm coordinates)")
                    print(f"Total traces in figure: {len(fig.data)}")
            else:
                print("\n⚠ No neuron compartments to plot")
        else:
            print("⚠ Neurons not available for plotting")
    else:
        print("⚠ No synapses found for these neurons")
else:
    print("⚠ Skipping synapse visualization - neurons not loaded from previous section")

**Requirements:**

1. **Install caveclient** (already in our runtime script):
   ```bash
   pip install caveclient
   ```

2. **BANC Authentication (optional):** While some data is publicly accessible, you may need a BANC API token for full access:
   - Get a token from: https://global.daf-apis.com/auth/api/v1/create_token
   
   **For local setup:**
   - Save to `~/.cloudvolume/secrets/cave-secret.json`:
     ```json
     {"token": "YOUR_TOKEN_HERE"}
     ```
   
   **For Google Colab:**
   - Create the secrets directory and file:
     ```python
     import os
     import json
     
     # Create directory
     os.makedirs(os.path.expanduser('~/.cloudvolume/secrets'), exist_ok=True)
     
     # Save token (replace with your actual token)
     token_data = {"token": "YOUR_TOKEN_HERE"}
     with open(os.path.expanduser('~/.cloudvolume/secrets/cave-secret.json'), 'w') as f:
         json.dump(token_data, f)
     
     print("✓ CAVE token saved")
     ```

Let's download and visualize some EPG neurons from the BANC dataset:

In [ ]:
# Check if caveclient is available
# Note: No need for meshparty! navis.patch_cloudvolume provides mesh downloading directly
try:
    import caveclient
    CAVE_AVAILABLE = True
    print(f"✓ caveclient version: {caveclient.__version__}")
except ImportError as e:
    CAVE_AVAILABLE = False
    print(f"✗ caveclient not installed: {e}")
    print("  Install with: pip install caveclient")
    print("  Skipping Extension 03...")

if CAVE_AVAILABLE:
    print()
    print("="*70)
    print("Extension 03: Downloading and Visualizing BANC Meshes")
    print("="*70)

    # Step 1: Get EPG neuron IDs from metadata
    print("\n[1/3] Finding EPG neurons in metadata...")
    epg_mask = meta_full['cell_type'].str.contains('EPG', case=False, na=False)
    epg_meta = meta_full[epg_mask]
    print(f"✓ Found {len(epg_meta)} EPG neurons")

    # Use first 2 EPG neurons for visualization
    epg_ids = epg_meta[DATASET_ID].values[:2].tolist()  # Convert to list for cloudvolume
    print(f"  Will download {len(epg_ids)} meshes")

    # Step 2: Set up navis for CloudVolume mesh downloading
    print("\n[2/3] Setting up CloudVolume integration...")
    try:
        # Patch navis to add CloudVolume methods
        navis.patch_cloudvolume()
        print("✓ navis.patch_cloudvolume() applied")

        # Connect to BANC and get CloudVolume object
        client = caveclient.CAVEclient('brain_and_nerve_cord')
        vol = client.info.segmentation_cloudvolume()
        print(f"✓ Connected to {client.datastack_name}")

    except Exception as e:
        print(f"✗ Failed to set up CloudVolume: {e}")
        print("  You may need to set up authentication:")
        print("  Get a token from: https://global.daf-apis.com/auth/api/v1/create_token")
        print("  Save to ~/.cloudvolume/secrets/cave-secret.json")
        CAVE_AVAILABLE = False

    if CAVE_AVAILABLE:
        # Step 3: Download meshes directly as NeuronList
        print(f"\n[3/3] Downloading {len(epg_ids)} EPG neuron meshes...")
        print("  (This may take a minute...)")

        try:
            # Download meshes - returns navis.NeuronList directly!
            banc_meshes = vol.mesh.get_navis(epg_ids)

            print(f"\n✓ Successfully downloaded {len(banc_meshes)} meshes")
            for i, mesh in enumerate(banc_meshes, 1):
                print(f"  [{i}] {mesh.name}: {mesh.n_vertices:,} vertices")

        except Exception as e:
            print(f"✗ Failed to download meshes: {e}")
            banc_meshes = []

        # Visualize with navis.plot3d()
        if len(banc_meshes) > 0:
            print("\nCreating visualization...")

            # Generate rainbow colors for each mesh
            colors = plt.cm.rainbow(np.linspace(0, 1, len(banc_meshes)))
            color_list = [tuple(c[:3]) for c in colors]

            # Plot with navis
            fig = navis.plot3d(
                banc_meshes,
                color=color_list,
                backend='plotly',
                width=1000,
                height=800,
                title=f'{len(banc_meshes)} EPG Neuron Meshes from BANC'
            )

            if fig is not None:
                fig.update_layout(
                    scene=dict(
                        xaxis_title='X (nm)',
                        yaxis_title='Y (nm)',
                        zaxis_title='Z (nm)',
                        aspectmode='data'
                    )
                )
                fig.show()
                print("✓ Visualization complete!")
            else:
                print("✗ Failed to create visualization")
        else:
            print("\n✗ No meshes downloaded - cannot create visualization")


---

## Summary

In this tutorial, we covered:

1. **Loading Neuron Skeletons**: Reading SWC files from GCS with navis
2. **3D Visualization**: Interactive plotting with plotly backend
3. **Morphometric Analysis**: Extracting cable length, branch counts, etc.
4. **Neuropil Meshes**: Visualizing neurons in anatomical context
5. **NBLAST**: Morphological similarity comparison
6. **Template Brains**: Introduction to flybrains for spatial transformations

### Key navis Features

- **Simple API**: `navis.read_swc()`, `navis.plot3d()`, `navis.nblast()`
- **Fast**: Compiled code (Rust) for NBLAST and other operations
- **Flexible**: Multiple backends (plotly, octarine, k3d)
- **Integrated**: Works with pandas, numpy, and other scientific Python tools
- **Well-documented**: Extensive [documentation](https://navis.readthedocs.io/) and [tutorials](https://navis-org.github.io/navis/)

### Useful navis Functions

```python
# Reading neurons
navis.read_swc('neuron.swc')
navis.read_swc('neurons.zip', include_subdirs=True)

# Morphometrics
n.cable_length  # Total cable length
n.n_branches    # Number of branches
n.n_nodes       # Number of nodes
navis.segment_analysis(n)  # Detailed per-segment metrics

# Visualization
navis.plot3d(neurons, backend='plotly', color='blue')
navis.plot3d(neurons, backend='octarine')  # Modern, fast backend

# Morphological comparison
dotprops = navis.make_dotprops(neurons, k=5)
scores = navis.nblast(dotprops, dotprops)

# Spatial transformations
navis.xform_brain(neuron, source='BANC', target='JRC2018F')

# Manipulation
neurons / 1000  # Scale (e.g., nm to µm)
navis.downsample_neuron(n, factor=5)  # Reduce resolution
navis.prune_by_strahler(n, to_prune=1)  # Prune terminal branches
```

---

## Next Steps

- Explore Tutorial 03 for connectivity analysis
- Try different datasets (FAFB, MANC, etc.)
- Experiment with different NBLAST parameters
- Use flybrains to compare neurons across datasets
- Check out the [navis documentation](https://navis.readthedocs.io/) for more advanced features

---

**Tutorial complete!** 🎉

## Session Information

In [ ]:
import sys
import platform

print("Python Session Information")
print("=" * 50)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print("\nPackage Versions:")
print(f"  navis: {navis.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")

try:
    import flybrains
    print(f"  flybrains: {flybrains.__version__}")
except:
    print(f"  flybrains: not installed")

print("\n" + "=" * 50)